In [1]:
require 'nn'
require 'rnn'

In [68]:
-- Some useful functions
function genNbyK(n, k, a, b)
    out = torch.LongTensor(n, k)
    for i=1, n do
        for j = 1, k do
            out[i][j] = torch.random(a, b)
        end
    end
    return out
end

function buildModel(model, vocabSize, embeddingSize, metric, adapt, use_cuda)
    -- Small experiments seem to show that the Tanh activations performed better\
    --      than the ReLU for the bow model
    if model == 'bow' then
        print(string.format("Running bag-of-words model to learn %s", metric))
        sentenceLookup = nn.Sequential()
                    :add(nn.LookupTableMaskZero(vocabSize, embeddingSize))
                    :add(nn.Sum(2, 3, true)) -- Not averaging blows up model so keep this true
                    :add(nn.Tanh())
    else
        print(string.format("Running LSTM model to learn %s", metric))
        sentenceLookup = nn.Sequential()
                    :add(nn.LookupTableMaskZero(vocabSize, embeddingSize))
                    :add(nn.SplitTable(2))
                    :add(nn.Sequencer(nn.LSTM(embeddingSize, embeddingSize)))
                    :add(nn.SelectTable(-1))            -- selects last state of the LSTM
                    :add(nn.Linear(embeddingSize, embeddingSize))
                    :add(nn.ReLU())
    end
    local queryLookup = sentenceLookup:clone("weight", "gradWeight") 
    local summaryLookup = sentenceLookup:clone("weight", "gradWeight")
    local pmodule = nn.ParallelTable()
                :add(sentenceLookup)
                :add(queryLookup)
                :add(summaryLookup)

    if model == 'bow' then
        nnmodel = nn.Sequential()
            :add(pmodule)
            :add(nn.JoinTable(2))
            :add(nn.Tanh())
            :add(nn.Linear(embeddingSize * 3, 2))
    else
        nnmodel = nn.Sequential()
            :add(pmodule)
            :add(nn.JoinTable(2))
            :add(nn.ReLU())
            :add(nn.Linear(embeddingSize * 3, 2))
    end

    if adapt then 
        print("Adaptive regularization")
        local logmod = nn.Sequential()
            :add(nn.Linear(embeddingSize * 3, 1))
            :add(nn.LogSigmoid())
            :add(nn.SoftMax())

        local regmod = nn.Sequential()
            :add(nn.Linear(embeddingSize * 3, 2))

        local fullmod = nn.ConcatTable()
            :add(regmod)
            :add(logmod)

        local final = nn.Sequential()
            :add(pmodule)
            :add(nn.JoinTable(2))
            :add(fullmod)

        nnmodel = final
    end

    if use_cuda then
        return nnmodel:cuda()
    end
    return nnmodel
end

function Tokenize(inputdic)
    --- This function tokenizes the words into a unigram dictionary
    local out = {}

    for k, v in pairs(inputdic) do
        if out[v] == nil then
            out[v] = 1
        else 
            out[v] = 1 + out[v]
        end
    end
    return out
end

function rougeScores(genSummary, refSummary)
    local genTotal = 0
    local refTotal = 0
    local intersection = 0
    -- Inserting the missing keys
    for k, genCount in pairs(genSummary) do
        if refSummary[k] == nil then
            refSummary[k] = 0
        end
    end
    for k, refCount in pairs(refSummary) do
        local genCount = genSummary[k]
        if genCount == nil then 
            genCount = 0 
        end
        intersection = intersection + math.min(refCount, genCount)
        refTotal = refTotal + refCount
        genTotal = genTotal + genCount
    end

    recall = intersection / refTotal
    prec = intersection / genTotal
    if refTotal == 0 then
        recall = 0
    end 
    if genTotal == 0 then
        prec = 0
    end
    -- tmp = {intersection, refTotal, genTotal}
    if recall > 0 or prec > 0 then
        f1 = (2 * recall * prec) / (recall + prec)
    else 
        f1 = 0
    end
    return recall, prec, f1
end

In [ ]:
function buildPredsummary(summary, chosenactions, inputsentences, select_index)
    if summary == nil then
        summary = torch.zeros(inputsentences:size())
    end
    for i=1, chosenactions:size(1) do
        -- the 2 is for the SELECT index, will have to make this more general later
        if chosenactions[i][select_index] == 1 then
            summary[i]:copy(inputsentences[i])
        end
    end    
    return summary
end

function buildPredsummaryFast(summary, chosenactions, inputsentences, select_index)
    n = inputsentences:size(1)
    k = inputsentences:size(2)
    if summary == nil then
        summary = torch.zeros(inputsentences:size())
    end
    actionmatrix = chosenactions:select(2, select_index):clone():resize(n, 1):view(n, 1):expand(n, k):clone()
    --     This line didn't work for whatever reason...gives weird indexing...
    --     actionmatrix = chosenactions:select(2, select_index):resize(1, n):view(n, 1):expand(n, k):clone()
    return actionmatrix:cmul(inputsentences:double())
end

In [ ]:
function buildTotalSummary(predsummary, totalPredsummary)
    nps = predsummary:size(1)
    n_l = totalPredsummary:size(2)
    indices = torch.linspace(1, n_l, n_l):long() 
    for i=1, predsummary:size(1) do
        if predsummary[i]:sum() > 0 then 
            maxindex = 0
            for j = 1, totalPredsummary[i]:size(1) do 
                if totalPredsummary[i][j] == 0 then
                    maxindex = maxindex + 1
                end
            end
            lenx = predsummary[i]:size(1)
            totalPredsummary[i][{{maxindex - lenx + 1, maxindex}}]:copy(predsummary[i])
        end
    end
end

function buildTotalSummaryFast(predsummary, totalPredsummary)
    nps = predsummary:size(1)
    n_l = totalPredsummary:size(2)
    indices = torch.linspace(1, n_l, n_l):long() 
    for i=1, predsummary:size(1) do
        if predsummary[i]:sum() > 0 then 
            -- Finding the largest index with a zero
            maxindex = torch.max(indices[torch.eq(totalPredsummary[i], 0)])
            lenx = predsummary[i]:size(1)
            totalPredsummary[i][{{maxindex - lenx + 1, maxindex}}]:copy(predsummary[i])
        end
    end
end

In [3]:
-- Setting parameters
n = 10
n_s = 5
k = 7
q = 5
a = 1
b = 100
embDim = 50
SKIP = 1
SELECT = 2
fast = false

# Simulating the data

In [65]:
-- Simulating streams and queries
queries = genNbyK(n, q, a, b)

-- Note that the sentences are batched by sentence index so sentences[1] is the first sentence of each article
sentences = {}
for i=1, n_s do
    sentences[i] = genNbyK(n, k, a, b)
end

-- Optimal predicted summary
trueSummary = torch.zeros(n, k * n_s)
-- Using this to generate the optimal actions
true_actions = {}
for i=1, n_s do 
    ---- Simulating the data
    trueqValues = torch.rand(n, 2)
    
     ---- Generating the max values and getting the indices
    qMaxtrue, qindxtrue = torch.max(trueqValues, 2)
    
    --- I want to select the qindx elements for each row
    true_actions[i] = torch.zeros(n, 2):scatter(2, qindxtrue, torch.ones(trueqValues:size()))
    best_sentences = buildPredsummaryFast(best_sentences, true_actions[i], sentences[i], SELECT)
    buildTotalSummaryFast(best_sentences, trueSummary)
end

qTokens = {}
for i=1, n do
    qTokens[i] = Tokenize(trueSummary[i]:totable())
end

# Scoring the rougue metrics on the simulation

In [1]:
print(rougeScores(Tokenize(trueSummary[1]:totable()), Tokenize(trueSummary[1]:totable())))

1	1	1	


# Building the model

In [26]:
model = buildModel('bow', b, embDim, 'f1', false, false)

Running bag-of-words model to learn f1	


# Scoring the model on the sentences

In [45]:
totalPredsummary = torch.zeros(n, n_s * k)

for i=1, n_s do 
    preds = model:forward({sentences[i], queries, totalPredsummary})
    if fast then 
        qMax, qindx = torch.max(preds, 2)  -- Pulling the best actions
        -- Here's the fast way to select the optimal action for each query
        actions = torch.zeros(n, 2):scatter(2, qindx, torch.ones(preds:size()))
        predsummary = buildPredsummaryFast(predsummary, actions, sentences[i], SELECT)
        buildTotalSummaryFast(predsummary, totalPredsummary)
    else 
        actions = torch.zeros(n, 2)
        for j=1, n do
            if preds[j][SELECT] > preds[j][SKIP] then
                actions[j][SELECT] = 1
            else
                actions[j][SKIP] = 1
            end
        end
        predsummary = buildPredsummary(predsummary, actions, sentences[i], SELECT)
        buildTotalSummary(predsummary, totalPredsummary)       
    end
end

In [ ]:
function stackMemory(newinput, memory_hist, memsize, adapt, use_cuda)
    local sentMemory = torch.cat(newinput[1][1]:double(), memory_hist[1][1]:double(), 1)
    local queryMemory = torch.cat(newinput[1][2]:double(), memory_hist[1][2]:double(), 1)
    local sumryMemory = torch.cat(newinput[1][3]:double(), memory_hist[1][3]:double(), 1)
    local rewardMemory = torch.cat(newinput[2]:double(), memory_hist[2]:double(), 1)

    if adapt then
        regMemory = torch.cat(newinput[4]:double(), memory_hist[4]:double(), 1)
    end 

    if use_cuda then 
        actionMemory = torch.cat(newinput[3]:double(), memory_hist[3]:double(), 1)
    else 
        actionMemory = torch.cat(newinput[3], memory_hist[3], 1)
    end
    --- specifying rows to index 
    if sentMemory:size(1) <= memsize then
        nend = sentMemory:size(1)
        nstart = 1
    else 
        nstart = math.max(memsize - sentMemory:size(1), 1)
        nend = memsize + nstart
    end
    --- Selecting n last data points
    sentMemory = sentMemory[{{nstart, nend}}]
    queryMemory = queryMemory[{{nstart, nend}}]
    sumryMemory = sumryMemory[{{nstart, nend}}]
    rewardMemory = rewardMemory[{{nstart, nend}}]
    actionMemory = actionMemory[{{nstart, nend}}]

    if use_cuda then
        inputMemory = {sentMemory:cuda(), queryMemory:cuda(), sumryMemory:cuda()}
        rewardMemory = rewardMemory:cuda()
        actionMemory = torch.ByteTensor(#actionMemory):copy(actionMemory):cuda()
    end

    inputMemory = {sentMemory, queryMemory, sumryMemory}
    if adapt then
        regMemory = regMemory[{{nstart, nend}}]
        return {inputMemory, rewardMemory, actionMemory, regMemory}
    end 
    return {inputMemory, rewardMemory, actionMemory}
end    